In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [ ]:
def f(x, t):
    a = x[0]
    b = x[1]
    c = x[2]
    d = x[3]
    return a * np.exp(-b*t) * np.cos(c * t + d)

x = [2, 3, 30, -0.75]
t = np.linspace(0, 1, 50)
np.random.seed(450)
y = f(x, t)+0.05*np.random.randn(len(t))
plt.plot(t, y, 'o')
np.savez('data.npz', t=t, y=y)

## Load data

Load data $t_i$ and $y_i$.  Then plot it.

In [ ]:
data = np.load('data.npz')
t = data['t']
y = data['y']
plt.plot(t, y, 'o')

## Solving the model in 3 ways

In the following you will implement two ways to determine parameters $a$, $b$, $c$, and $d$ in the model
$$
m(t) = a e^{-b t} \cos(c t + d)
$$

1. Using `scipy.optimize.least_squares`
2. Constructing your own Gauss-Newton method (Using `np.linalg.lstsq`)

To do this, you will need at least three functions:

In [ ]:
def model(x, t):
    a = x[0]
    b = x[1]
    c = x[2]
    d = x[3]
    return a * np.exp(-b*t) * np.cos(c * t + d)

def residual(x, t, y):
    return y - model(x, t)

def J(x, t, y):
    a = x[0]
    b = x[1]
    c = x[2]
    d = x[3]
    return np.array([         -np.exp(-b*t) * np.cos(c * t + d),
                     a * t * np.exp(-b*t) * np.cos(c * t + d),
                     a * t * np.exp(-b*t) * np.sin(c * t + d),
                     a *     np.exp(-b*t) * np.sin(c * t + d)]).T

## Find an estimate

Find an estimate to parameters $a$, $b$, $c$, and $d$.

In [ ]:
x0 = np.array([1, 1, 1, 1])
result = least_squares(residual, x0, jac=J, args=(t,y), verbose=2)
print(result.x)

In [ ]:
T = np.linspace(t.min(), t.max(), 200)
x = result.x
plt.plot(T, model(x, T))
plt.plot(T, model([2, 3, 30, -0.75], T), 'r:')
plt.plot(t, y, 'o')
print(x)

In [ ]:
x = np.array([2, 3, 30, -0.7])

In [ ]:
for _ in range(20):
    x = x + np.linalg.lstsq(J(x, t, y), -residual(x, t, y))[0]
plt.plot(t, y, 'o')
plt.plot(T, model(x, T), 'r-')